# Implementation of collaborative filtering on a recommendation system 

This note is intended to demonstrate how collaborative filtering algorithms is implemented in recommendation systems. 
Instead using conjugate gradient, here we used alternating least squares (ALS) to solve the problem.

At the beginning, we assume there are 5 users and 10 movies. The rating matrix will be 10x6, where each element is 1-10, a rating score. The element (i,j) denotes the rating score of movie-i by user-j.

In [253]:
import numpy as np
import math
from numpy import *

num_movies =10
num_users = 5

ratings = random.randint(11, size = (num_movies, num_users))
print (ratings)

[[7 6 9 3 1]
 [1 5 1 4 7]
 [6 3 9 1 9]
 [6 9 9 4 6]
 [0 8 3 2 6]
 [7 2 4 6 2]
 [3 5 5 2 6]
 [7 3 8 3 6]
 [7 9 6 8 4]
 [1 2 6 6 3]]


In [254]:
did_ratings = (ratings != 0)*1
print (did_ratings)

[[1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [0 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]]


Now, suppose we have a new user, Hsiang. This user's rating is represented by a 10-component rating vector. We can first initialize the null rating vector:

In [255]:
hsiang_rating = zeros((num_movies,1))
print (hsiang_rating)

[[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]]


Instead, we can also designate the rating vector by generating random numbers:

In [256]:
hsiang_ratings = random.randint(11,size =(num_movies,1))
print (hsiang_ratings)

[[0]
 [1]
 [2]
 [3]
 [8]
 [3]
 [7]
 [3]
 [9]
 [1]]


Then we merge the new user's rating in the rating matrix. Since now there are 6 users, we can see there are six columns in the rating matrix.

In [257]:
ratings = append(hsiang_ratings, ratings, axis =1)
did_ratings = append((hsiang_ratings!=0)*1, did_ratings, axis = 1)
print (np.around(ratings,2))
print (did_ratings)

[[0 7 6 9 3 1]
 [1 1 5 1 4 7]
 [2 6 3 9 1 9]
 [3 6 9 9 4 6]
 [8 0 8 3 2 6]
 [3 7 2 4 6 2]
 [7 3 5 5 2 6]
 [3 7 3 8 3 6]
 [9 7 9 6 8 4]
 [1 1 2 6 6 3]]
[[0 1 1 1 1 1]
 [1 1 1 1 1 1]
 [1 1 1 1 1 1]
 [1 1 1 1 1 1]
 [1 0 1 1 1 1]
 [1 1 1 1 1 1]
 [1 1 1 1 1 1]
 [1 1 1 1 1 1]
 [1 1 1 1 1 1]
 [1 1 1 1 1 1]]


In the above, the first matrix is the rating matrix. The second one is the matrix, whose (i,j) element is 1 if user-j has rated movie-i, whereas = 0 if the users didn't rate the movie-i. Note that we used the convention: rating 1-10. In the rating matrix, rating =0 means "the user has not rated the movie yet"; the data is missing.


## Mean-normalization

Next step we need to mean normalize the rating matrix.

In [258]:
import numpy as np
def normalize_ratings(ratings, did_ratings):
    num_movies = ratings.shape[0]
    ratings_mean = [0]*num_movies
    ratings_norm = zeros(shape = ratings.shape)
    for i in range(num_movies):
        idx = where(did_ratings[i] == 1)[0]
        ratings_mean[i] = np.mean(ratings[i,idx])
        ratings_norm[i,idx] = ratings[i, idx]- ratings_mean[i]
    return ratings_norm, ratings_mean

#print (ratings, did_ratings)
ratings, ratings_mean = normalize_ratings(ratings, did_ratings)
print (np.around(ratings,1))

[[ 0.   1.8  0.8  3.8 -2.2 -4.2]
 [-2.2 -2.2  1.8 -2.2  0.8  3.8]
 [-3.   1.  -2.   4.  -4.   4. ]
 [-3.2 -0.2  2.8  2.8 -2.2 -0.2]
 [ 2.6  0.   2.6 -2.4 -3.4  0.6]
 [-1.   3.  -2.   0.   2.  -2. ]
 [ 2.3 -1.7  0.3  0.3 -2.7  1.3]
 [-2.   2.  -2.   3.  -2.   1. ]
 [ 1.8 -0.2  1.8 -1.2  0.8 -3.2]
 [-2.2 -2.2 -1.2  2.8  2.8 -0.2]]


After mean-normalizing, the positive values indicate the rating beyond the average, the negative values mean "users don't like the movies". The exact "0" mean the movie has not been rated by the users yet.

In [259]:
num_users = ratings.shape[1]
num_movies = ratings.shape[0]
print (num_users, num_movies)
num_features = 3

6 10


"num_features =3" means there are three latent factors; could be "action", "romance", "comedy" or any others. These latent factors describe features of movies. We only recommend movies to users who have similar taste on movies, or they may like the movie.

To be more concrete, for example, if a movie "Warcraft" has a high score on action by many users, it means this movie has strong preference on action. If user-1, say, Alice, doesn't like action movie, meaning that she possibly will give low rating to Warcraft". Then obviously we won't recommend the action movie Alice. On the other hand, if the user-2, Joe, always rates high score on action movies, we know he may like "Warcraft". Even though Joe has not watched yet, we can recommend "Warcraft" to Joe.

Remind again. Aftet mean-normalization, positive values suggest positive preference, whereas negative ones mean the user dislikes. Exact "0"'s means the users have not watched yet.
 


## Initialize the movies' features and users' preference

How can we find the latent factors? Using linear regression. 

We fisrt consider that the rating matrix can be decomposed as two matrices, Rating = U * V.
For simple linear regression problems, y=theta*x, we know (x,y) and try to find out Theta to best describe the correlation between (x,y). 

In our matrix problem, we can think about each element in the rating matrix is a rating[i,j] = y, theta is the attributes for each movie-i, x is user-j's preference. In reality, theta and x are unknown. In the collaborative filtering algorithm, we initially guess the values of Theta and X:

In [265]:
movies_features = random.randn(nums_movies, nums_features)
users_preference = random.randn(nums_users, nums_features)
print (movies_features)

[[-0.99633304 -0.66022008 -1.03651279]
 [-0.85943883  1.15216689  2.18023273]
 [ 0.11363454  0.01710109 -0.32018263]
 [-0.38804045 -2.43522192 -0.37708437]
 [-0.11813036  1.71939601 -0.45814617]
 [ 0.45819133 -0.92005296 -1.24911749]
 [ 0.41449486  0.27185492 -0.27395849]
 [-1.11926801 -1.2589305  -0.48007282]
 [ 0.83475855  0.02389139 -0.47600654]
 [ 0.54152236  0.31649974  1.10087383]]


The j-th row of "movies_features" denotes the j-th movie's features. For example, for j=1, X1 = (x11, x12, x13) means the movie has feature "action"=x11, "romance"=x12, "comedy"=x13. Here we will denote it as U.

In [266]:
print (users_preference)

[[-0.34034663  2.61657888  1.37143206]
 [ 0.89372139 -0.23251664  1.00898648]
 [-0.53687364  0.24529631  0.78791333]
 [-1.15833453  0.23128601  0.19295087]
 [ 0.04698052  0.32548204 -0.41042209]
 [ 0.56284968 -0.11649399 -0.4150762 ]]


The i-th row of "users_preference" denotes the i-th user's preference, Theta(i) = (ti1, ti2, ti3) means the level of  action, romance and comedy movies the user-i likes. Hereafter we will denote it as V.

In [262]:
initial_Theta_X = r_[movies_features.T.flatten(), users_preference.T.flatten()]
print(initial_Theta_X)

[ 1.08808386 -0.71489339 -2.20505238  0.32832656  1.4422673   2.78588963
 -0.71141742  0.43186069 -0.20470045  0.76426419  0.09406951  0.48134434
 -0.25078608  2.31938796  0.41265807  2.15990492 -0.32095182 -0.3960826
  1.91751958  0.55523168 -2.87927849  0.1737703   0.13369125  0.81648228
 -0.66734808 -0.13872669  0.50599594  0.35043516  0.00412928  0.15324317
  1.45034913  0.85189741 -0.62059825 -0.72950777 -1.26273711 -0.94952881
 -1.03228682 -0.60630743 -0.47535134  0.12417856 -0.59128421  0.1514741
  1.71986978  0.08041668 -1.12398661 -0.14544704 -0.36398353  0.74393368]


"initial_Theta_X" is a matrix (X* Theta^T) with dimension of (num_movies * num_features) * (num_features x * num_users). Here X is 10x6 = (10x3)*(3*6). So we have X = U (V.T).


## Alternating least squares

Instead using the conjugate gradient, we implement alternating least squares (ALS) algorithm. We need to solve X = U*V.T, but both U and V are unknown. After initially guessing U and V, we can:

(1) fixed V, to find U

(2) fixed U, to find V

(3) repeat above procedures, until U*V.T doesn't change much.

This is the ALS. In the following we use normal equation to solve U and V respectively.

In [267]:
def errorCheck(ratings, U, T):
        num_movies = ratings.shape[0]
        num_users = ratings.shape[1]
        err =0
        n = 0
        for movie in range(num_movies):
            for user in range(num_users):
                err += (ratings[movie, user] - (U[movie]*(V.T)[user].T)[0,0])**2 
                n+=1
        return np.sqrt(err/n)
       
        
lamda =0.3
U = np.mat(movies_features)
epsilo = 0.00000001
err = 1000
for iter in range(100):
        # compute movies' features
        newV = np.linalg.inv(U.T * U + lamda* np.mat(np.eye(3))) * U.T* ratings
        V = newV ## now V is updated
        
        # compute users features
        newU = np.linalg.inv(V * V.T + lamda* np.mat(np.eye(3))).T * V * np.mat(ratings).T
        U = newU.T  ## now U is updated
        
        err_update = errorCheck(ratings, U, V)
        print (iter, err, err_update)
        if abs ((err-err_update)/err) < epsilo: break
        err = err_update

print (' ----- Now ALS is convergent -----')     
print ('MSE =', err)  

0 1000 1.07117666017
1 1.07117666017 0.924362358499
2 0.924362358499 0.904897992687
3 0.904897992687 0.900511012914
4 0.900511012914 0.898954944292
5 0.898954944292 0.898160378141
6 0.898160378141 0.89764753544
7 0.89764753544 0.897275390852
8 0.897275390852 0.896990114249
9 0.896990114249 0.896765013625
10 0.896765013625 0.896584093442
11 0.896584093442 0.89643668548
12 0.89643668548 0.896315257429
13 0.896315257429 0.896214309214
14 0.896214309214 0.896129729845
15 0.896129729845 0.896058389458
16 0.896058389458 0.895997867652
17 0.895997867652 0.89594626684
18 0.89594626684 0.895902081037
19 0.895902081037 0.895864101864
20 0.895864101864 0.895831350116
21 0.895831350116 0.895803025191
22 0.895803025191 0.895778467236
23 0.895778467236 0.895757128438
24 0.895757128438 0.895738551027
25 0.895738551027 0.895722350212
26 0.895722350212 0.895708200842
27 0.895708200842 0.895695826852
28 0.895695826852 0.895684992856
29 0.895684992856 0.895675497392
30 0.895675497392 0.895667167445
31 0.

Now U and V are convergently computed after 86 iterations. The movie feature matrix U is

In [268]:
print (np.around(U,2))

[[-0.46 -0.83 -1.83]
 [-0.22  0.67  1.61]
 [-2.07 -0.98  0.58]
 [-1.09 -0.45 -0.27]
 [-0.71  1.6  -0.6 ]
 [ 0.82 -1.   -0.33]
 [-0.83  0.83 -0.16]
 [-0.99 -1.09 -0.07]
 [ 0.78  0.51 -0.91]
 [ 0.43 -1.16  0.71]]


The fifth movie has highr rating on the second features than others; meaning it is a romance movie.
The V matrix is

In [269]:
print (np.around(V,2))

[[ 0.6  -0.11 -0.23 -1.24  2.35 -1.37]
 [ 1.58 -0.8   1.29 -1.98 -0.6   0.51]
 [-1.06 -0.98 -0.48 -0.58  0.95  2.15]]


We can see the first user like romance since the preference is higher than others.
So we can predict the first user will give 5-th movie very high-raitng:

In [270]:
print (np.around(U*V,2))

[[ 0.37  2.51 -0.09  3.25 -2.31 -3.74]
 [-0.79 -2.09  0.14 -1.97  0.61  4.1 ]
 [-3.4   0.43 -1.07  4.17 -3.72  3.59]
 [-1.08  0.74 -0.21  2.4  -2.54  0.69]
 [ 2.73 -0.61  2.53 -1.94 -3.2   0.49]
 [-0.73  1.04 -1.32  1.15  2.21 -2.35]
 [ 0.98 -0.42  1.34 -0.53 -2.58  1.21]
 [-2.23  1.05 -1.15  3.43 -1.75  0.65]
 [ 2.24  0.41  0.92 -1.44  0.65 -2.77]
 [-2.32  0.18 -1.94  1.35  2.38  0.36]]


In [271]:
print (np.around(ratings))

[[ 0.  2.  1.  4. -2. -4.]
 [-2. -2.  2. -2.  1.  4.]
 [-3.  1. -2.  4. -4.  4.]
 [-3. -0.  3.  3. -2. -0.]
 [ 3.  0.  3. -2. -3.  1.]
 [-1.  3. -2.  0.  2. -2.]
 [ 2. -2.  0.  0. -3.  1.]
 [-2.  2. -2.  3. -2.  1.]
 [ 2. -0.  2. -1.  1. -3.]
 [-2. -2. -1.  3.  3. -0.]]


As we see. Now we can use the collaborative filtering to fill the missing data, and recommend new movies to users.